## Image Captionning

### Imports

In [1]:
import torch, sys, torchvision
from PIL import Image
from dataloader.get import DataGetter
sys.path.append('./LAVIS/')
from lavis.models import load_model_and_preprocess

In [2]:
dg=DataGetter()
print(torch.cuda.is_available())

True


In [3]:
# setup device to use
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
print(device)

# we associate a model with its preprocessors to make it easier for inference.
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="large_coco", is_eval=True, device=device
)
# uncomment to use base model
# model, vis_processors, _ = load_model_and_preprocess(
#     name="blip_caption", model_type="base_coco", is_eval=True, device=device
# )
vis_processors.keys()

def get_caption(image,device):
    
    raw_image = Image.fromarray(image)
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    
    # we'll first use beam search to generate a caption
    caption = model.generate({"image": image})
    
    # we'll then use nucleus sampling to get multiple captions
    # and more precise captions (you can uncomment the line below)
    
    # caption = model.generate({"image": image}, use_nucleus_sampling=True, num_captions=3)
    
    # store the generated caption in a variable
    return caption


    

cuda


In [4]:
# import matplotlib.pyplot as plt


# def caption_from_image_file(x):
#     return [str(get_caption(i,device)) for i in x.load()]

# import cv2
# import numpy as np


# df = dg.getData("train")

# df_test = df.head(5)

# # # start timer  
# # import time
# # start_time = time.time()


# # df_test['captions'] = df_test.images.apply(caption_from_image_file)

# # # end timer (in minutes)
# # print("--- %s minutes ---" % ((time.time() - start_time)/60))

# # df_test.to_csv('test.csv',index=False)

# # # free up cuda memory
# torch.cuda.empty_cache()

# # df_test.captions



# import dask.dataframe as dd

# # Create a Dask dataframe from a Pandas dataframe
# df_dask = dd.from_pandas(df_test, npartitions=4)

# # Apply the `caption_from_image_file` function to the `images` column using the `map` method
# df_dask['captions'] = df_dask['images'].map(caption_from_image_file, meta=('captions', 'f8'))

# torch.cuda.empty_cache()

# # Convert the Dask dataframe back to a Pandas dataframe
# df_result = df_dask.compute()




# # import cv2
# # import numpy as np

# # df = dg.getData("train")
# # df_test = df.head(5)

# # # start timer
# # import time
# # start_time = time.time()

# # def caption_from_image_file(img):
# #     return str(get_caption(img, device))

# # import multiprocessing as mp
# # from multiprocessing import Pool

# # pool = Pool(mp.cpu_count())
# # df_test['captions'] = pool.map(caption_from_image_file, df_test.images)

# # print("--- %s minutes ---" % ((time.time() - start_time)/60))

# # df_test.to_csv('test.csv',index=False)















In [5]:
# import sys
# sys.path.append("/opt/spark-3.2.1-bin-hadoop3.2/python")
# sys.path.append("/opt/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.4-src.zip")
# import findspark
# findspark.init()
# findspark.find()
# from pyspark.sql import SparkSession

# import os
# os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-19"
# os.environ["SPARK_HOME"] = "C:/Program Files/Spark/spark-3.3.1-bin-hadoop3"

# def get_captions_spark(df):
#     # Create a Spark session
#     spark = SparkSession.builder.getOrCreate()
    
#     print("Spark session created")
    
#     # Create a DataFrame from the input df
#     images_df = spark.createDataFrame(df)
    
#     # Split the DataFrame into batches of size 50
#     images_rdd = images_df.rdd.mapPartitions(lambda x: [x[1:] for x in x]).flatMap(lambda x: x).zipWithIndex().map(lambda x: (x[1], x[0]))
#     captions_rdd = images_rdd.map(lambda x: (x[0], [get_caption(im, device) for im in x[1].load()]))

#  # Convert the RDD back to a DataFrame
#     captions_df = captions_rdd.toDF(["index", "captions"])
    
#     # Join the DataFrames
#     result_df = images_df.join(captions_df, "index").drop("index")
    
#     return result_df.toPandas()
 
# df = dg.getData("train")
# df_test = df.head(5)

# print(df_test.dtypes)

# # delete all object columns except the images column
# object_cols = df_test.select_dtypes(include=['object']).columns.tolist()
# object_cols.remove('images')
# df_test = df_test.drop(object_cols, axis=1)

# # start timer
# import time
# start_time = time.time()
# df_test = get_captions_spark(df_test)
# print("--- %s minutes ---" % ((time.time() - start_time)/60))




In [6]:
import asyncio

async def get_caption_async(image):
    captions = get_caption(image, device)
    return captions

async def get_captions_batch_async(row, device):
    tasks = []
    for im in row.load():
        tasks.append(get_caption_async(im))
    captions = await asyncio.gather(*tasks)
    return captions



async def get_column_for_captions_async(ds, batch_size=50):
    captions = []
    all_captions=[]
    dj=ds['images']
    
    for i in range(len(dj)):
        batch_captions = await get_captions_batch_async(dj.iloc[i], device)
        all_captions.append(batch_captions)
    
    return all_captions




if __name__ == '__main__':
    df=DataGetter()
    df=df.getData('train')
    ds=df[:100]
    all_captions = asyncio.run(get_column_for_captions_async(ds, batch_size=50))
    ds['captions']=all_captions
    print(ds)
    ds.to_csv('tab.csv')



RuntimeError: asyncio.run() cannot be called from a running event loop